In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os, shutil
import cv2
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline
plt.style.use('ggplot')

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout,BatchNormalization,Activation
from tensorflow.keras.applications.vgg19 import VGG19,preprocess_input
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [3]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print(cv2.__version__)

TensorFlow version: 2.16.1
4.10.0


In [3]:
train_datagen = ImageDataGenerator(rescale=1./255.,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   shear_range=0.4,
                                   width_shift_range=0.25,
                                   height_shift_range=0.25,
                                   rotation_range=45,
                                   fill_mode='nearest')

In [4]:
valid_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [6]:
train_generator = train_datagen.flow_from_directory('tumorous_and_nontumorous/train/', 
                                                    batch_size=32, 
                                                    target_size=(120,120), 
                                                    class_mode='categorical',
                                                    shuffle=True, 
                                                    seed = 42, 
                                                    color_mode = 'rgb')

Found 19674 images belonging to 2 classes.


In [7]:
valid_generator = train_datagen.flow_from_directory('tumorous_and_nontumorous/valid/', 
                                                    batch_size=32, 
                                                    target_size=(120,120), 
                                                    class_mode='categorical',
                                                    shuffle=True, 
                                                    seed = 42, 
                                                    color_mode = 'rgb')

Found 2162 images belonging to 2 classes.


In [8]:
test_generator = train_datagen.flow_from_directory('tumorous_and_nontumorous/test/', 
                                                   batch_size=32, 
                                                   target_size=(120,120), 
                                                   class_mode='categorical',
                                                   shuffle=False, 
                                                   seed = 42, 
                                                   color_mode = 'rgb')


Found 2158 images belonging to 2 classes.


In [9]:
nb_samples = test_generator.samples

In [10]:
print("True labels shape:", test_generator.classes.shape)
print("Unique true labels:", np.unique(test_generator.classes))

True labels shape: (2158,)
Unique true labels: [0 1]


In [11]:
from keras.models import load_model

vgg19_model = load_model('model_vgg19.keras')
densenet_model = load_model('model_DenseNet121.keras')
inceptionresnet_model = load_model('model_incepresnet.keras')
inceptionv3_model = load_model('model_Inceptionv3.keras')
xception_model = load_model('model_xception.keras')


In [13]:
import numpy as np

def predict_with_models(models, test_generator):
    predictions = []
    for model in models:
        preds = model.predict(test_generator, steps=test_generator.samples // test_generator.batch_size + 1)
        predictions.append(preds)
    return np.array(predictions)


In [14]:
def ensemble_predictions(predictions):
    return np.mean(predictions, axis=0)


In [15]:
models = [vgg19_model, densenet_model, inceptionresnet_model, inceptionv3_model, xception_model]

# Get individual model predictions
individual_preds = predict_with_models(models, test_generator)

# Ensemble the predictions
final_predictions = ensemble_predictions(individual_preds)

# Convert probabilities to class labels (assuming binary classification)
final_labels = np.argmax(final_predictions, axis=1)


C:\Users\abhi2\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


68/68 ━━━━━━━━━━━━━━━━━━━━ 48s 703ms/step
68/68 ━━━━━━━━━━━━━━━━━━━━ 26s 336ms/step
68/68 ━━━━━━━━━━━━━━━━━━━━ 29s 370ms/step
68/68 ━━━━━━━━━━━━━━━━━━━━ 16s 207ms/step
68/68 ━━━━━━━━━━━━━━━━━━━━ 27s 383ms/step


In [16]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

# Get true labels from the test generator
true_labels = test_generator.classes  # Assuming this gives you the true labels

# Calculate accuracy
accuracy = accuracy_score(true_labels, final_labels)
print(f'Accuracy: {accuracy:.4f}')

# Generate confusion matrix
conf_matrix = confusion_matrix(true_labels, final_labels)

# Print classification report
print(classification_report(true_labels, final_labels,digits=4))

print(conf_matrix)


Accuracy: 0.9963
              precision    recall  f1-score   support

           0     0.9963    0.9963    0.9963      1079
           1     0.9963    0.9963    0.9963      1079

    accuracy                         0.9963      2158
   macro avg     0.9963    0.9963    0.9963      2158
weighted avg     0.9963    0.9963    0.9963      2158

[[1075    4]
 [   4 1075]]
